<a href="https://colab.research.google.com/github/viscio/NER_retrain/blob/main/NER_RISE_model_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.30 datasets torch accelerate optimum peft adapter-transformers seqeval
!pip install -i https://test.pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers[senten

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import platform
import torch

from datasets import load_dataset
from transformers import AutoModelForTokenClassification, BertTokenizerFast
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import load_metric

In [ ]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: Tesla V100-SXM2-16GB
Compute Capability: (7, 0)
Total Memory: 16935419904 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 5.15.120+
Python Version: 3.10.12


In [ ]:
# Load the MultiNERD dataset
dataset = load_dataset("Babelscape/multinerd")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Explore the structure of the dataset
print("Dataset structure:", dataset)

# Print some basic information
print("Dataset Description:", dataset["train"].info.description)
print("Dataset Citation:", dataset["train"].info.citation)
print("Dataset License:", dataset["train"].info.license)
print("Dataset Homepage:", dataset["train"].info.homepage)
print("Dataset Size:", dataset["train"].num_rows)
print("Dataset Features:", list(dataset["train"].features))

# Example of accessing a few data points
print("\nExample Data Points:")
for i in range(3):
    print(dataset['train'][i])

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 2678400
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 334800
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 335986
    })
})
Dataset Description: 
Dataset Citation: 
Dataset License: 
Dataset Homepage: 
Dataset Size: 2678400
Dataset Features: ['tokens', 'ner_tags', 'lang']

Example Data Points:
{'tokens': ['2002', 'ging', 'er', 'ins', 'Ausland', 'und', 'wechselte', 'für', '750.000', 'Pfund', 'Sterling', 'zu', 'Manchester', 'City', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0], 'lang': 'de'}
{'tokens': ['Die', 'Provinzhauptstadt', 'Burgos', 'ist', 'knapp', '80', 'km', '(', 'Fahrtstrecke', ')', 'in', 'nordwestlicher', 'Richtung', 'entfernt', ';', 'die', 'Stadt', 'Aranda', 'de', 'Duero', 'befindet', 'sich', 'ca.', '26', 'km', 'südwestlich', '.'], 'ne

In [ ]:
language_field = 'lang'

# Extract unique languages from the dataset
def get_unique_languages(dataset, language_field, sample_size=100000):
    unique_languages = set()
    for split in dataset.keys():
        subset = dataset[split].shuffle(seed=42).select(range(min(sample_size, len(dataset[split]))))
        for example in subset:
            if language_field in example:
                unique_languages.add(example[language_field])
    return unique_languages

# Get the unique languages
unique_languages = get_unique_languages(dataset, language_field)

# Print the unique languages
print(f"Unique Languages in the Dataset: {unique_languages}")

Unique Languages in the Dataset: {'es', 'it', 'pt', 'de', 'nl', 'fr', 'zh', 'en', 'pl', 'ru'}


In [ ]:
label_names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-ANIM", "I-ANIM", "B-BIO", "I-BIO",
               "B-CEL", "I-CEL", "B-DIS", "I-DIS", "B-EVE", "I-EVE", "B-FOOD", "I-FOOD", "B-INST", "I-INST",
               "B-MEDIA", "I-MEDIA", "B-MYTH", "I-MYTH", "B-PLANT", "I-PLANT", "B-TIME", "I-TIME", "B-VEHI", "I-VEHI",]

In [ ]:
language_field = 'lang'

# Filter for English examples
def filter_english(example):
    return example[language_field] == 'en'

english_dataset = dataset.filter(filter_english)

# Print out the size of the English subset for verification
for split in english_dataset.keys():
    print(f"Number of {split} samples in English: {len(english_dataset[split])}")

Filter:   0%|          | 0/2678400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/334800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

Number of train samples in English: 262560
Number of validation samples in English: 32820
Number of test samples in English: 32908


In [ ]:
label2id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }
id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-ANIM",
    8: "I-ANIM",
    9: "B-BIO",
    10: "I-BIO",
    11: "B-CEL",
    12: "I-CEL",
    13: "B-DIS",
    14: "I-DIS",
    15: "B-EVE",
    16: "I-EVE",
    17: "B-FOOD",
    18: "I-FOOD",
    19: "B-INST",
    20: "I-INST",
    21: "B-MEDIA",
    22: "I-MEDIA",
    23: "B-MYTH",
    24: "I-MYTH",
    25: "B-PLANT",
    26: "I-PLANT",
    27: "B-TIME",
    28: "I-TIME",
    29: "B-VEHI",
    30: "I-VEHI",
  }

In [ ]:
def load_ner_model(model_name, num_labels, id2label, label2id):
    # Load pre-trained model for token classification (NER)
    model = AutoModelForTokenClassification.from_pretrained(
        pretrained_model_name_or_path=model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
        low_cpu_mem_usage=True,
        )

    # Load the corresponding tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(
        model_name,
        model_max_length=512,
        )

    return model, tokenizer

model_name = "distilbert-base-uncased"
adapter_name = "AdapterHub/bert-base-uncased-pf-conll2003"
model, tokenizer = load_ner_model(model_name, len(id2label), id2label, label2id)

print(f"Loaded Model: {model}\nLoaded Tokenizer: {tokenizer}")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Loaded Model: DistilBertForTokenClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): Linear(
              in_features=768, out_fe

In [ ]:
#Get the values for input_ids, attention_mask, adjusted labels
def tokenize_align_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], padding="max_length", is_split_into_words=True, truncation=True)

  total_adjusted_labels = []

  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []

    for word_idx in word_ids_list:
      # Special tokens have a word id that is None. We set the label to -100 so they are automatically
      # ignored in the loss function.
      if(word_idx is None):
        adjusted_label_ids.append(-100)
      elif(word_idx!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = word_idx
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)

  #add adjusted labels to the tokenized samples
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_english_dataset = english_dataset.map(tokenize_align_labels, batched=True, remove_columns=['tokens', 'ner_tags', 'lang'])

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

cuda


DistilBertForTokenClassification(
  (shared_parameters): ModuleDict()
  (distilbert): DistilBertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): Linear(
              in_features=768, out_features=768, bi

In [ ]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-14-1176731c1857>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    fp16=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_english_dataset["train"],
    eval_dataset=tokenized_english_dataset["validation"],
    #peft_config=peft_config,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 262560
  Num Epochs = 2
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 2
  Total optimization steps = 10940
  Number of trainable parameters = 66386719
You're using a BertTokenizerFast tokenizer. Please note that with a fast 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.054300,0.071588,0.862592,0.878518,0.870482,0.973593


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.054300,0.071588,0.862592,0.878518,0.870482,0.973593
2,0.033600,0.071920,0.875187,0.884689,0.879912,0.975150


Saving model checkpoint to ./results/checkpoint-6000
Configuration saved in ./results/checkpoint-6000/config.json
Model weights saved in ./results/checkpoint-6000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-6000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-6000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-6500
Configuration saved in ./results/checkpoint-6500/config.json
Model weights saved in ./results/checkpoint-6500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-6500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-6500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-7000
Configuration saved in ./results/checkpoint-7000/config.json
Model weights saved in ./results/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-7000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-7000/special

TrainOutput(global_step=10940, training_loss=0.06325044448894382, metrics={'train_runtime': 3889.5778, 'train_samples_per_second': 135.007, 'train_steps_per_second': 2.813, 'total_flos': 6.864453116264448e+16, 'train_loss': 0.06325044448894382, 'epoch': 2.0})

In [ ]:
# Create a folder in the root directory
!mkdir -p "/content/drive/My Drive/tuned_models"
trainer.save_model("/content/drive/My Drive/tuned_models/fine_tuned_model_A")

Saving model checkpoint to /content/drive/My Drive/tuned_models/fine_tuned_model_A
Configuration saved in /content/drive/My Drive/tuned_models/fine_tuned_model_A/config.json
Model weights saved in /content/drive/My Drive/tuned_models/fine_tuned_model_A/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/tuned_models/fine_tuned_model_A/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/tuned_models/fine_tuned_model_A/special_tokens_map.json


In [ ]:
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from transformers import Trainer

def align_predictions(predictions, label_ids, label_map):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore special tokens
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(label_map[label_ids[batch_idx][seq_idx]])
                example_preds.append(label_map[preds[batch_idx][seq_idx]])
        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

def evaluate_model(trainer, tokenized_dataset, label_map):
    # Predictions on the test dataset
    predictions, label_ids, _ = trainer.predict(tokenized_dataset["test"])

    # Align predictions with the actual labels
    true_labels, predicted_labels = align_predictions(predictions, label_ids, label_map)

    # Calculate metrics
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    # Detailed classification report
    report = classification_report(true_labels, predicted_labels)

    return precision, recall, f1, report

precision, recall, f1, report = evaluate_model(trainer, tokenized_english_dataset, id2label)

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 32908
  Batch size = 24


In [ ]:
def report_to_df(report):
    # Split the report into lines and filter out empty lines
    report_lines = [line.strip() for line in report.split('\n') if line.strip()]

    # Initialize lists to hold the parsed data
    rows = []

    # Parse lines
    for line in report_lines[2:-3]:  # Skip the header and overall statistics
        line_parts = [part for part in line.split('  ') if part]
        entity, precision, recall, f1_score, support = line_parts
        rows.append({
            'Entity': entity.strip(),
            'Precision': float(precision.strip()),
            'Recall': float(recall.strip()),
            'F1-Score': float(f1_score.strip()),
            'Support': int(support.strip())
        })

    # Create a DataFrame
    df = pd.DataFrame(rows)

    return df

# Example usage
report_df = report_to_df(report)

In [ ]:
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

Precision:  0.9288106550354389
Recall:  0.909379766141332
F1:  0.9189925119128659


In [ ]:
report_df

,Entity,Precision,Recall,F1-Score,Support
0,BIO,0.53,0.70,0.60,46
1,CEL,0.77,0.70,0.73,114
2,DIS,0.81,0.76,0.78,3742
3,EVE,0.93,0.91,0.92,886
4,FOOD,0.60,0.64,0.62,1796
5,INST,0.65,0.54,0.59,48
6,LOC,0.97,0.97,0.97,36206
7,MEDIA,0.95,0.92,0.94,1196
8,MYTH,0.76,0.71,0.73,116
9,ORG,0.96,0.95,0.95,9412
